In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

In [19]:
torch.__version__

'2.5.1+cpu'

In [20]:
import torch
print(torch.backends.mkldnn.is_available())  # True if MKL-DNN is active


True


##1. Data Preproecessing 

In [21]:
# Define data transformations for data augmentation and normalization
data_transforms = {
    'train_set': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test_set': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [22]:
data_dir='dataset'
print(data_transforms.keys())  # Should output dict_keys(['train_set', 'test_set'])



dict_keys(['train_set', 'test_set'])


In [23]:
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train_set'), data_transforms['train_set'])
print(len(train_dataset))  # Should print the number of images in train_set
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test_set'), data_transforms['test_set'])
print(len(test_dataset))  # Should print the number of images in test_set

class_names = train_dataset.classes
print(class_names)


8000
2000
['cats', 'dogs']


In [24]:
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)  # Using the updated 'weights' argument

for name, param in model.named_parameters():
    if "fc" in name:  # Unfreeze the final classification layer
        param.requires_grad = True
    else:
        param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Use all parameters


# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

device

device(type='cpu')

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Create a dictionary for dataloaders
dataloaders = {
    'train_set': train_loader,
    'test_set': test_loader
}

# Get dataset sizes
dataset_sizes = {
    'train_set': len(train_dataset),
    'test_set': len(test_dataset)
}

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for phase in ['train_set', 'test_set']:
        if phase == 'train_set':
            model.train()  # Set model to training mode
        else:
            model.eval()  # Set model to evaluation mode

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train_set'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train_set':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("Training complete!")


In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')


In [ ]:
torch.save(model.state_dict(), 'dog_cat_model.pth')  # Save the model


In [27]:
# Recreate the model architecture
from torchvision.models import resnet18, ResNet18_Weights
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)  # Ensure you're using the same model architecture
for name, param in model.named_parameters():
    if "fc" in name:  # Unfreeze the final classification layer
        param.requires_grad = True
    else:
        param.requires_grad = False

# Load the saved weights with weights_only=True to avoid executing arbitrary code
model.load_state_dict(torch.load('dog_cat_model.pth', map_location=device), strict=False)
model = model.to(device)

# Now, the model is ready to use


C:\Users\jeevan\AppData\Local\Temp\ipykernel_4524\2455816289.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('dog_cat_model.pth', map_l

In [31]:
from PIL import Image

def predict(image_path, model, class_names):
    model.eval()
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)
    return class_names[pred]

image_path = 'cat_dog.jpg'
prediction = predict(image_path, model, class_names)
print(f'Predicted Class: {prediction}')


Predicted Class: dogs
